In [ ]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [ ]:
TRAIN_CSV = '../input/quora-question-pairs/train.csv'
TEST_CSV = '../input/quora-question-pairs/test.csv'
EMBEDDING_FILE = '../input/kowordvec/ko.bin'
#MODEL_SAVING_DIR = '/home/ecohen/HDD/HDD4/Models/Kaggle/Quora/'
#train_df = pd.read_csv('../input/quora-question-pairs/train.csv')

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
stops = set(stopwords.words('english'))
train_df

In [ ]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [ ]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True, encoding='utf8',unicode_errors='ignore')
# from gensim.models.wrappers import FastText
# word2vec = FastText.load_fasttext_format(EMBEDDING_FILE)

questions_cols = ['question1', 'question2']

In [ ]:
# tokenize
c = 0
for question in tqdm(questions):
    questions[c] = list(gensim.utils.tokenize(question, deacc=True, lower=True))
    c += 1
print("ok")
# train model
model = gensim.models.Word2Vec(questions, size=300, workers=-1, iter=10, negative=20)
print("ok2")
# trim memory
model.init_sims(replace=True)

In [ ]:
questions_cols = ['question1', 'question2']

In [ ]:
model.init_sims(replace=True)
word2vec = model

In [ ]:
# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                
                if word in stops and word not in word2vec.vocabulary:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)

In [ ]:
# save model
import warnings
warnings.filterwarnings("ignore")
model.save('3_word2vec.mdl')
model.wv.save_word2vec_format('3_word2vec.bin', binary=True)

In [ ]:
import spacy

In [ ]:
# exctract word2vec vectors
import spacy
nlp = spacy.load('en')
print(model.wv.get_vector('trump'))

vecs1 = [doc.vector for doc in nlp.pipe(train_df['question1'], n_threads=-1, batch_size=20)]
vecs1 =  np.array(vecs1)
print("ok")
train_df['q1_feats'] = list(vecs1)
    
vecs2 = [doc.vector for doc in nlp.pipe(train_df['question2'], n_threads=-1, batch_size=20)]
vecs2 =  np.array(vecs2)
train_df['q2_feats'] = list(vecs2)

    # save features
pd.to_pickle(df, '1_df.pkl')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(train_df['question1']) + list(train_df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
#word2tfidf
#w2v.keys()
questions

In [ ]:
train_df

In [ ]:
# spacy를 대체함
#from konlpy.tag import Kkma

In [ ]:
#twitter = Twitter()
#twitter.pos("What is the story of Kohinoor (Koh-i-Noor) Diamond?")

In [ ]:
# exctract word2vec vectors
#import spacy
#nlp = spacy.load('en')

vecs1 = []
for qu in tqdm(list(train_df['question1'])):
    doc = twitter.pos(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # print(word)
        # word2vec
        
        try:
            vec = model.wv.get_vector(word[0])
        except KeyError as ex:
            #print(ex)
            continue
            
        # fetch df score
        
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
            
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs1.append(mean_vec)
train_df['q1_feats'] = list(vecs1)
train_df['q1_feats']

In [ ]:

vecs2 = []
for qu in tqdm(list(train_df['question2'])):
    doc = twitter.pos(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # print(word)
        # word2vec
        
        try:
            vec = model.wv.get_vector(word[0])
        except KeyError as ex:
            #print(ex)
            continue
            
        # fetch df score
        
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
            
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs2.append(mean_vec)
train_df['q2_feats'] = list(vecs2)
train_df['q2_feats']

In [ ]:
qu = "안녕하세요. 딥러닝 초보자입니다"
doc = twitter.pos(qu) 
train_df.head()

In [ ]:
doc

In [ ]:
# shuffle df
train_df = train_df.reindex(np.random.permutation(train_df.index))

# set number of train and test instances
num_train = int(train_df.shape[0] * 0.88)
num_test = train_df.shape[0] - num_train                 
print("Number of training pairs: %i"%(num_train))
print("Number of testing pairs: %i"%(num_test))

# init data data arrays
X_train = np.zeros([num_train, 2, 300])
X_test  = np.zeros([num_test, 2, 300])
Y_train = np.zeros([num_train]) 
Y_test = np.zeros([num_test])

# format data 
b = [a[None,:] for a in list(train_df['q1_feats'].values)]
q1_feats = np.concatenate(b, axis=0)

b = [a[None,:] for a in list(train_df['q2_feats'].values)]
q2_feats = np.concatenate(b, axis=0)

# fill data arrays with features
X_train[:,0,:] = q1_feats[:num_train]
X_train[:,1,:] = q2_feats[:num_train]
Y_train = train_df[:num_train]['is_duplicate'].values
            
X_test[:,0,:] = q1_feats[num_train:]
X_test[:,1,:] = q2_feats[num_train:]
Y_test = train_df[num_train:]['is_duplicate'].values

# remove useless variables
del b
del q1_feats
del q2_feats

In [ ]:
from siamese import *
X_train[0]

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Lambda, merge, BatchNormalization, Activation, Input, Merge
from keras import backend as K
from keras.layers import normalization


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_dim):
    '''
    Base network for feature extraction.
    '''
    input = Input(shape=(input_dim, ))
    # print(input)
    dense1 = Dense(128)(input)
    #bn1 = BatchNormalization(mode=2)(dense1)
    relu1 = Activation('relu')(dense1)

    dense2 = Dense(128)(relu1)
    #bn2 = BatchNormalization(mode=2)(dense2)
    res2 = merge([relu1, dense2], mode='sum')
    relu2 = Activation('relu')(res2)    

    dense3 = Dense(128)(relu2)
    #bn3 = BatchNormalization(mode=2)(dense3)
    res3 = Merge(mode='sum')([relu2, dense3])
    relu3 = Activation('relu')(res3)   
    
    feats = merge([relu3, relu2, relu1], mode='concat')
    # bn4 = BatchNormalization(mode=2)(feats)

    model = Model(input=input, output=feats)

    return model


def compute_accuracy(predictions, labels):
    '''
    Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def create_network(input_dim):
    # network definition
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    return model

In [ ]:
# create model
from keras.optimizers import RMSprop, SGD, Adam
net = create_network(300)

# train
#optimizer = SGD(lr=1, momentum=0.8, nesterov=True, decay=0.004)
optimizer = Adam(lr=0.001)
net.compile(loss=contrastive_loss, optimizer=optimizer)

for epoch in range(50):
    net.fit([X_train[:,0,:], X_train[:,1,:]], Y_train,
          validation_data=([X_test[:,0,:], X_test[:,1,:]], Y_test),
          batch_size=128, nb_epoch=1, shuffle=True, )
    
    # compute final accuracy on training and test sets
    pred = net.predict([X_test[:,0,:], X_test[:,1,:]], batch_size=128)
    te_acc = compute_accuracy(pred, Y_test)
    
#    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))